In [1]:
import torch
print(torch.cuda.is_available())  # Should be True if CUDA is available
print(torch.cuda.current_device())  # Print the index of the current GPU
print(torch.cuda.get_device_name(0))  # Print the name of the GPU

True
0
NVIDIA GeForce RTX 3070 Laptop GPU


In [2]:
import torch
import os
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import pandas as pd
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import DataFrameLoader
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

from dotenv import load_dotenv
load_dotenv()

True

In [3]:
ACCESS_TOKEN = os.getenv("ACCESS_TOKEN")

model_id = "google/gemma-2b-it"
tokenizer = AutoTokenizer.from_pretrained(model_id, token=ACCESS_TOKEN)
quantization_config = BitsAndBytesConfig(load_in_4bit=True,
                                         bnb_4bit_use_double_quant=True,
                                         bnb_4bit_compute_dtype=torch.bfloat16)

model = AutoModelForCausalLM.from_pretrained(model_id,
                                             device_map="auto",
                                             quantization_config=quantization_config,
                                             token=ACCESS_TOKEN)
model.eval()
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"using {device}")

`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

using cuda


In [4]:
def inference(question: str, context: str):

    if context == None or context == "":
        prompt = f"""Tell me you didn't get the context and can't answer the question: {question}"""
    else:
        prompt = f"""You are a Russian-speaking news correspondent. People will ask you questions about the news.
        Use the following pieces of context to answer the question at the end. Please follow the following rules:
        1. If you don't know the answer, don't try to make up an answer. Just say "I can't find the final answer but you may want to check the following links".
        2. If you find the answer, write the answer in a concise way with five sentences maximum.
        3. Give the answer on Russian.
        Context: {context}.
        Question: {question}
        """
    chat = [
        {"role": "user", "content": prompt},
    ]
    formatted_prompt = tokenizer.apply_chat_template(
        chat,
        tokenize=False,
        add_generation_prompt=True,
    )
    inputs = tokenizer.encode(
        formatted_prompt, add_special_tokens=False, return_tensors="pt"
    ).to(device)
    with torch.no_grad():
        outputs = model.generate(
            input_ids=inputs,
            max_new_tokens=250,
            do_sample=False,
        )
    response = tokenizer.decode(outputs[0], skip_special_tokens=False)
    response = response[len(formatted_prompt) :]
    response = response.replace("<eos>", "")
    return response

question = "Tell me the latest news"
print(inference(question=question, context=""))

C:\Users\ekurochkina\IdeaProjects\NLPTasksProject_env_store\LentaNewsRAGProjct\Lib\site-packages\transformers\models\gemma\modeling_gemma.py:540: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


I do not have access to real-time information and cannot provide the latest news. For the most up-to-date news and updates, I recommend checking reputable news sources or search engines.


In [6]:
news_df = pd.read_csv('../lenta-ru-news-cropped.csv')
loader = DataFrameLoader(news_df, page_content_column='text')
documents = loader.load()

text_splitter = CharacterTextSplitter(chunk_size = 700, chunk_overlap = 50)
docs = text_splitter.split_documents(documents)

huggingface_embeddings_ru = HuggingFaceEmbeddings(model_name='cointegrated/rubert-tiny2')

C:\Users\ekurochkina\AppData\Local\Temp2\ipykernel_48012\2837384794.py:8: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  huggingface_embeddings_ru = HuggingFaceEmbeddings(model_name='cointegrated/rubert-tiny2')
C:\Users\ekurochkina\IdeaProjects\NLPTasksProject_env_store\LentaNewsRAGProjct\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [8]:
vectorstore = FAISS.load_local('../faiss_full_index_character_1k_ru', huggingface_embeddings_ru,  allow_dangerous_deserialization=True )

In [9]:
question = "Новость про пожар в Санкт-Петербурге"
retrieved_docs = vectorstore.similarity_search(question, k=5)
context = "".join(doc.page_content + "\n" for doc in retrieved_docs)
print(context)

Около двух часов ночи с понедельника на вторник в жилой квартире на первом этаже дома 117 корпус 1 по Московскому проспекту в Петербурге возник пожар. Как сообщили РИА "Новости" со ссылкой на пресс-службу Главного управления государственной противопожарной службы МВД РФ, находившиеся в квартире двое мужчин и три женщины погибли, скорее всего, из-за отравления дымом. Огонь захватил площадь в 20 квадратных метров в кухне и коридоре. По информации ИТАР-ТАСС, тушением занимались 2 пожарных расчета. Через час огонь был локализован. На место прибыли три бригады "скорой помощи". Из горящей квартиры удалось выбраться только ее хозяйке. Причины пожара пока не установлены, отметили в пресс-службе МВД. По предварительным результатам расследования, пожар произошел из-за неосторожного обращения с огнем подвыпивших обитателей квартиры.
Как сообщила телекомпания НТВ, 23 сентября около 15:50 по местному времени оперативному дежурному ГУВД Санкт-Петербурга поступила информация о взрыве в жилом доме на 

In [10]:
torch.cuda.empty_cache()
print(inference(question=question, context=context))

Около двух часов ночи с понедельника на вторник в жилом квартире на первом этаже дома 117 корпус 1 по Московскому проспекту в Петербурге возник пожар. Как сообщили РИА "Новости" со ссылкой на пресс-службу Главного управления государственной противопожарной службы МВД РФ, находившиеся в квартире двое мужчин и три женщины погибли, скорее всего, из-за отравления дымом.
